# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Strong: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 
  - Boss: Implement [Self-Attention Pooling](https://arxiv.org/pdf/2008.01077v1.pdf) & [Additive Margin Softmax](https://arxiv.org/pdf/1801.05599.pdf) to further boost the performance.

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/ac77388c90204a4c8daebeddd40ff916)
  - Slide: [link](https://docs.google.com/presentation/d/1HLAj7UUIjZOycDe7DaVLSwJfXVd3bXPOyzSb6Zk3hYU/edit?usp=sharing)
  - Data: [link](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

# Download dataset
- Data is [here](https://drive.google.com/drive/folders/1vI1kuLB-q1VilIftiwnPOCAeOOFfBZge?usp=sharing)

## Fix Random Seed

In [1]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(2023)
kfold = 8

# Data

## Dataset
- Original dataset is [Voxceleb2](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/vox2.html).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb2.
- We randomly select 600 speakers from Voxceleb2.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory  
  - data directory  
  |---- metadata.json  
  |---- testdata.json  
  |---- mapping.json  
  |---- uttr-{random string}.pt

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

class VoxDataset(Dataset):
    def __init__(self, data_dir, segment_len=512):
        self.training = False
        self.data_dir = data_dir
        self.segment_len = segment_len
        self.p = 0.95
        print(f"dataset configs: segment_len={segment_len}, p={self.p}")
    
        # Load the mapping from speaker neme to their corresponding id. 
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]
    
        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]
    
        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
    def __len__(self):
            return len(self.data)
 
    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        if len(mel) > self.segment_len:
            start = random.randint(0, len(mel) - self.segment_len)
            mel = torch.FloatTensor(mel[start:start + self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
        
        if self.training is True:
            seglen, speclen = mel.shape
            ptime = random.uniform(0, self.p)
            pspec = random.uniform(0, self.p)
            time_mask_len = int(seglen * ptime)
            time_mask_beg = random.randint(0, seglen - time_mask_len)
            spec_mask_len = int(speclen * pspec)
            spec_mask_beg = random.randint(0, speclen)
            mel[time_mask_beg:time_mask_beg + time_mask_len, :] = 0
            if spec_mask_beg + spec_mask_len < speclen:
                mel[:, spec_mask_beg:spec_mask_beg + spec_mask_len] = 0
            else:
                mel[:, spec_mask_beg:speclen] = 0
                mel[:, 0:spec_mask_len - (speclen - spec_mask_beg)] = 0
        
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker
 
    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.

In [3]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = VoxDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    proportions = [1. / kfold for i in range(kfold)]
    lengths = [int(p * len(dataset)) for p in proportions]
    lengths[-1] = len(dataset) - sum(lengths[:-1])
    sets = random_split(dataset, lengths, generator=torch.Generator().manual_seed(2023))

    for trainset in sets:
        trainset.training = True
    train_loader = [DataLoader(
            trainset,
            batch_size=batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=n_workers,
            pin_memory=True,
            collate_fn=collate_batch,
        ) for trainset in sets]

    for validset in sets:
        validset.training = False
    valid_loader = [DataLoader(
            validset,
            batch_size=batch_size,
            num_workers=n_workers,
            drop_last=True,
            pin_memory=True,
            collate_fn=collate_batch,
        ) for validset in sets]

    return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - embed: the number of expected features of the input (required).
    - nhead: the number of heads of the multiheadattention models (required).
    - dim_feedforward: the dimension of the feedforward network model (default=2048).
    - dropout: the dropout value (default=0.1).
    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).
    - num_layers: the number of sub-encoder-layers in the encoder (required).
    - norm: the layer normalization component (optional).

In [4]:
## import torch
import torch.nn as nn
import torch.nn.functional as F


class SubSampling(nn.Module):
    def __init__(self, in_channel=1, out_channel=3):
        super(SubSampling, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
        )
    def forward(self, x):
        outputs = self.model(x.unsqueeze(1))
        batch_size, c, seqlen, width = outputs.size()
        outputs = outputs.permute(0, 2, 1, 3)
        outputs = outputs.contiguous().view(batch_size, seqlen, c * width)
        return outputs

class Perm(nn.Module):
    def __init__(self, order):
        super(Perm, self).__init__()
        self.order = order
    def forward(self, x):
        return x.permute(*self.order)

class FFN(nn.Module):
    def __init__(self, embed, dropout):
        super(FFN, self).__init__()
        # inout: (batch_size, seqlen, embed)
        self.model = nn.Sequential(
            nn.LayerNorm(embed),
            nn.Linear(embed, embed * 4),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(embed * 4, embed),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        out = self.model(x)
        return out + x

class PointwiseConv(nn.Module):
    def __init__(self, in_channel, out_channel):
        # (batch, c, seq)
        super(PointwiseConv, self).__init__()
        self.model = nn.Conv1d(in_channel, out_channel, kernel_size=1, stride=1, padding=0)
    def forward(self, x):
        return self.model(x)

class DepthwiseConv(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size):
        # (batch, c, seq)
        print(f"depwise_conv config: kernel_size={kernel_size}")
        super(DepthwiseConv, self).__init__()
        assert out_channel % in_channel == 0
        padding = (kernel_size - 1) // 2
        self.model = nn.Conv1d(in_channel, out_channel, kernel_size=kernel_size,
                               stride=1, padding=padding, groups=in_channel)
    def forward(self, x):
        return self.model(x)

class ConvBlock(nn.Module):
    def __init__(self, embed, dropout):
        super(ConvBlock, self).__init__()
        # inout: (batch, seq, embed)
        self.model = nn.Sequential(
            nn.LayerNorm(embed),                                  # out: (batch, seq, embed)
            Perm((0, 2, 1)),                                        # out: (batch, embed, seq)
            PointwiseConv(in_channel=embed, out_channel=embed * 2), # out: (batch, embed * 2, seq)
            nn.GLU(dim=1),
            DepthwiseConv(in_channel=embed, out_channel=embed, kernel_size=9),     # inout: (batch, embed, seq)
            nn.BatchNorm1d(embed),
            nn.SiLU(),
            PointwiseConv(in_channel=embed, out_channel=embed),
            nn.Dropout(dropout),
            Perm((0, 2, 1))                                         # out: (batch, seq, embed)
        )
    def forward(self, x):
        out = self.model(x)
        return out + x

class MultiHeadAttention(nn.Module):
    def __init__(self, embed, num_heads, dropout):
        super(MultiHeadAttention, self).__init__()
        self.layernorm = nn.LayerNorm(embed)
        self.attention = nn.MultiheadAttention(embed, num_heads, dropout=dropout, batch_first=True)
    def forward(self, x, mask=None):
        xx = self.layernorm(x)
        out, _ = self.attention(xx, xx, xx, need_weights=False)
        return out + x

class ConformerBlock(nn.Module):
    def __init__(self, embed, dropout):
        super(ConformerBlock, self).__init__()
        self.model = nn.Sequential(
            FFN(embed=embed, dropout=dropout),
            MultiHeadAttention(embed=embed, num_heads=8, dropout=dropout),
            ConvBlock(embed=embed, dropout=dropout),
            FFN(embed=embed, dropout=dropout),
            nn.LayerNorm(embed)
        )
    def forward(self, x):
        return self.model(x)

class AdditiveMarginSoftmax(nn.Module):
    def __init__(self, embed, n_spks):
        super(AdditiveMarginSoftmax, self).__init__()
        self.margin = 0.35
        self.factor = 30
        self.n_spks = n_spks
        self.W = torch.nn.Parameter(torch.randn(embed, n_spks), requires_grad=True)
        nn.init.xavier_normal_(self.W, gain=1)
    def forward(self, x, y=None):
        norm_x = torch.norm(x, dim=1, p=2, keepdim=True).clamp(min=1e-12)
        x_norm = torch.div(x, norm_x)
        norm_W = torch.norm(self.W, dim=0, p=2, keepdim=True).clamp(min=1e-12)
        W_norm = torch.div(self.W, norm_W)
        out = torch.mm(x_norm, W_norm)
        sub = torch.zeros_like(out)
        if (y != None):
            sub = sub.scatter_(1, y.view(-1, 1), self.margin)
        out = self.factor * (out - sub)
        return out

class SelfAttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(input_dim, 1)
    def forward(self, batch_rep):
        att_w = F.softmax(self.W(batch_rep).squeeze(-1), dim=-1).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)
        return utter_rep

class Classifier(nn.Module):
    def __init__(self, feat=40, embed=256, n_spks=600, dropout=0.3):
        print(f"running config: embed={embed}, dropout={dropout}")
        super().__init__()
        subsample_expand = 4
        subsample_feat = (feat + 1) // 2
        
        self.model = nn.Sequential(
            SubSampling(in_channel=1, out_channel=subsample_expand), # out: (batch, seq, feat * channel)
            nn.Linear(subsample_feat * subsample_expand, embed),
            #nn.Linear(feat, embed),
            nn.Dropout(dropout),
            ConformerBlock(embed=embed, dropout=dropout),
            SelfAttentionPooling(embed),                   # out: (batch, embed)
            nn.LayerNorm(embed),
            nn.Linear(embed, embed)
        )
        self.output = AdditiveMarginSoftmax(embed, n_spks)
        print(f"AM-Softmax config: margin={self.output.margin}, factor={self.output.factor}")
    def forward(self, mels, y=None):
                                        # in:  (batch, seq, feat)
        out = self.model(mels)          # out: (batch, n_spks)
        spaced = self.output(out, y)        
        raw = self.output(out, None)
        return spaced, raw

# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [5]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
    Create a schedule with a learning rate that decreases following the values of the cosine function between the
    initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
    initial lr set in the optimizer.

    Args:
        optimizer (:class:`~torch.optim.Optimizer`):
        The optimizer for which to schedule the learning rate.
        num_warmup_steps (:obj:`int`):
        The number of steps for the warmup phase.
        num_training_steps (:obj:`int`):
        The total number of training steps.
        num_cycles (:obj:`float`, `optional`, defaults to 0.5):
        The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
        following a half-cosine).
        last_epoch (:obj:`int`, `optional`, defaults to -1):
        The index of the last epoch when resuming training.

    Return:
        :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
    """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))

    return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [6]:
import torch


def model_fn(batch, model, criterion, device, training=False):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs, raw = model(mels, labels)

    if training:
        loss = criterion(outs, labels)
    else:
        loss = criterion(raw, labels)

    # Get the speaker id with highest probability.
    preds = raw.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [7]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.4f}",
            accuracy=f"{running_accuracy / (i+1):.4f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)

# Main function

In [8]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
    """arguments"""
    config = {
        "data_dir": "/kaggle/input/ml2022spring-hw4/Dataset",
        "save_path": "model",
        "batch_size": 32,
        "n_workers": 0,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "total_steps": 70000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
    running_patch = 0
    train_iterator = iter(train_loader[running_patch])
    print(f"[Info]: Finish loading data!",flush = True)

    model = [Classifier(n_spks=speaker_num).to(device) for i in range(kfold)]
    criterion = nn.CrossEntropyLoss()
    optimizer = [AdamW(model[i].parameters(), lr=1e-3) for i in range(kfold)]
    scheduler = [get_cosine_schedule_with_warmup(optimizer[i], warmup_steps, total_steps) for i in range(kfold)]
    print(f"[Info]: Finish creating model!",flush = True)

    best_accuracy = [-1.0 for i in range(kfold)]

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            running_patch = (running_patch + 1) % kfold
            train_iterator = iter(train_loader[running_patch])
            batch = next(train_iterator)
        for i in range(kfold):
            if i == running_patch:
                continue
            loss, accuracy = model_fn(batch, model[i], criterion, device, training=True)
            batch_loss = loss.item()
            batch_accuracy = accuracy.item()
            loss.backward()
            optimizer[i].step()
            scheduler[i].step()
            optimizer[i].zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
            running_patch=running_patch
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            for i in range(kfold):
                valid_accuracy = valid(valid_loader[i], model[i], criterion, device)

                # keep the best model
                if valid_accuracy > best_accuracy[i]:
                    best_accuracy[i] = valid_accuracy
                    torch.save(model[i].state_dict(), f"{save_path}_{i}.ckpt")
                    
            print(best_accuracy)
            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
dataset configs: segment_len=512, p=0.95
[Info]: Finish loading data!
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: mar

Train: 100% 2000/2000 [11:06<00:00,  3.00 step/s, accuracy=0.53, loss=12.14, running_patch=1, step=2000]
Valid: 100% 7072/7083 [00:07<00:00, 898.71 uttr/s, accuracy=0.4337, loss=2.5714]
Valid: 100% 7072/7083 [00:08<00:00, 876.41 uttr/s, accuracy=0.4907, loss=2.2871]
Valid: 100% 7072/7083 [00:08<00:00, 871.94 uttr/s, accuracy=0.4104, loss=2.6835]
Valid: 100% 7072/7083 [00:08<00:00, 853.57 uttr/s, accuracy=0.4714, loss=2.4406]
Valid: 100% 7072/7083 [00:07<00:00, 894.08 uttr/s, accuracy=0.4941, loss=2.2681]
Valid: 100% 7072/7083 [00:08<00:00, 871.91 uttr/s, accuracy=0.4420, loss=2.5751]
Valid: 100% 7072/7083 [00:08<00:00, 871.21 uttr/s, accuracy=0.4726, loss=2.3596]
Valid: 100% 7072/7085 [00:08<00:00, 875.83 uttr/s, accuracy=0.4641, loss=2.4363]


[0.4336821266968326, 0.4906674208144796, 0.41035067873303166, 0.4714366515837104, 0.4940610859728507, 0.44202488687782804, 0.4725678733031674, 0.46408371040723984]


Train: 100% 2000/2000 [07:37<00:00,  4.38 step/s, accuracy=0.78, loss=8.51, running_patch=2, step=4000] 
Valid: 100% 7072/7083 [00:08<00:00, 858.34 uttr/s, accuracy=0.7040, loss=1.3173]
Valid: 100% 7072/7083 [00:07<00:00, 893.59 uttr/s, accuracy=0.7223, loss=1.2312]
Valid: 100% 7072/7083 [00:07<00:00, 889.08 uttr/s, accuracy=0.6739, loss=1.4515]
Valid: 100% 7072/7083 [00:08<00:00, 861.94 uttr/s, accuracy=0.7188, loss=1.2732]
Valid: 100% 7072/7083 [00:07<00:00, 893.99 uttr/s, accuracy=0.7288, loss=1.2325]
Valid: 100% 7072/7083 [00:08<00:00, 853.81 uttr/s, accuracy=0.6973, loss=1.3969]
Valid: 100% 7072/7083 [00:08<00:00, 880.65 uttr/s, accuracy=0.7149, loss=1.2646]
Valid: 100% 7072/7085 [00:08<00:00, 830.83 uttr/s, accuracy=0.7052, loss=1.3325]


[0.7040441176470589, 0.7222850678733032, 0.6739253393665159, 0.71875, 0.728789592760181, 0.6972567873303167, 0.7149321266968326, 0.7051753393665159]


...

Train: 100% 2000/2000 [07:35<00:00,  4.39 step/s, accuracy=1.00, loss=0.02, running_patch=3, step=68000]
Valid: 100% 7072/7083 [00:07<00:00, 926.35 uttr/s, accuracy=0.9519, loss=0.3182]
Valid: 100% 7072/7083 [00:07<00:00, 921.38 uttr/s, accuracy=0.9522, loss=0.3337]
Valid: 100% 7072/7083 [00:07<00:00, 907.92 uttr/s, accuracy=0.9490, loss=0.3260]
Valid: 100% 7072/7083 [00:07<00:00, 896.48 uttr/s, accuracy=0.9497, loss=0.3451]
Valid: 100% 7072/7083 [00:07<00:00, 920.02 uttr/s, accuracy=0.9488, loss=0.3666]
Valid: 100% 7072/7083 [00:07<00:00, 920.22 uttr/s, accuracy=0.9468, loss=0.3724]
Valid: 100% 7072/7083 [00:07<00:00, 919.66 uttr/s, accuracy=0.9546, loss=0.3224]
Valid: 100% 7072/7085 [00:08<00:00, 869.12 uttr/s, accuracy=0.9502, loss=0.3476]


[0.954185520361991, 0.9537613122171946, 0.950933257918552, 0.9496606334841629, 0.9492364253393665, 0.9468325791855203, 0.9568721719457014, 0.9502262443438914]


Train: 100% 2000/2000 [07:44<00:00,  4.31 step/s, accuracy=1.00, loss=0.08, running_patch=4, step=7e+4] 
Valid: 100% 7072/7083 [00:15<00:00, 449.81 uttr/s, accuracy=0.9491, loss=0.3314]
Valid: 100% 7072/7083 [00:14<00:00, 476.20 uttr/s, accuracy=0.9548, loss=0.3296]
Valid: 100% 7072/7083 [00:16<00:00, 434.84 uttr/s, accuracy=0.9490, loss=0.3336]
Valid: 100% 7072/7083 [00:11<00:00, 593.51 uttr/s, accuracy=0.9482, loss=0.3511]
Valid: 100% 7072/7083 [00:11<00:00, 618.81 uttr/s, accuracy=0.9512, loss=0.3515]
Valid: 100% 7072/7083 [00:16<00:00, 427.09 uttr/s, accuracy=0.9471, loss=0.3707]
Valid: 100% 7072/7083 [00:09<00:00, 756.37 uttr/s, accuracy=0.9559, loss=0.3168]
Valid: 100% 7072/7085 [00:08<00:00, 803.57 uttr/s, accuracy=0.9481, loss=0.3544]


[0.954185520361991, 0.9547511312217195, 0.950933257918552, 0.9496606334841629, 0.9512160633484162, 0.9471153846153846, 0.9568721719457014, 0.9502262443438914]


Train:   0% 0/2000 [00:00<?, ? step/s]


# Inference

## Dataset of inference

In [9]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        testdata_path = Path(data_dir) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        return feat_path, mel


def inference_collate_batch(batch):
    """Collate a batch of data."""
    feat_paths, mels = zip(*batch)

    return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [10]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
    """arguments"""
    config = {
        "data_dir": "/kaggle/input/ml2022spring-hw4/Dataset",
        "model_path": "./model",
        "output_path": "./output.csv",
    }

    return config


def main(
    data_dir,
    model_path,
    output_path,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=0,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!",flush = True)

    speaker_num = len(mapping["id2speaker"])
    model = [Classifier(n_spks=speaker_num) for i in range(kfold)]
    for i in range(kfold):
        model[i].load_state_dict(torch.load(f"{model_path}_{i}.ckpt"))
        model[i] = model[i].to(device)
        model[i].eval()
    print(f"[Info]: Finish creating model!",flush = True)

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            summation = None
            for md in model:
                outs, _ = md(mels)
                if summation == None:
                    summation = outs
                else:
                    summation += outs
            preds = summation.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
running config: embed=256, dropout=0.3
depwise_conv config: kernel_size=9
AM-Softmax config: margin=0.35, factor=30
[Info]: Finish creati

  0%|          | 0/8000 [00:00<?, ?it/s]